<a href="https://colab.research.google.com/github/Permanganant/Shazam_Song_Downloader/blob/main/Shazam_song_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube
!pip install youtube-search-python
!pip3 install ShazamAPI
!pip install ffmpeg-python > /dev/null
!pip install torchaudio
!pip install moviepy

In [ ]:
#Required Libraries

from pytube import YouTube
from pytube import Search
from ShazamAPI import Shazam
import json, requests 
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg
import tempfile
import pathlib
import torchaudio
import moviepy.editor as mp

In [ ]:
#To get audio for Colab this part of the code taken with some little modifications from https://ricardodeazambuja.com/deep_learning/2019/03/09/audio_and_video_google_colab/

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

 
  path = '/content/tmp.wav'
  with open(path, 'wb') as f:
      f.write(riff)
       
  x, sr = torchaudio.load(path)

  return x, sr
audio, sr = get_audio()

In [ ]:
mp3_file_content_to_recognize = open('tmp.wav', 'rb').read()
shazam = Shazam(mp3_file_content_to_recognize)
recognize_generator = shazam.recognizeSong()
try:
  youtube_json = next(recognize_generator)[1]['track']['sections'][2]['youtubeurl'] 
except ValueError:
  print("Oopss..Can't find song, Please Try Again")

In [ ]:
url = requests.get(youtube_json)
text = url.text
data = json.loads(text)
video_uri = data['actions'][0]['uri']
print("Video URI: " ,video_uri)

Video URI:  https://youtu.be/zn_axv6eoOw?autoplay=1


In [ ]:
yt = YouTube(video_uri)
len_query = len(yt.streams)
q = yt.streams
resolution = {'144':False,'240':False,'360':False,'480':False,'720':False,'1080':False,'1440':False,'2160':False}

for  i in range (len_query): 
  q = yt.streams[i]
  if q.resolution == '144p':
    resolution['144'] = True
  elif q.resolution == '240p':
    resolution['240'] = True
  elif q.resolution == '360p':
    resolution['360'] = True
  elif q.resolution == '480p':
    resolution['480'] = True
  elif q.resolution == '720p':
    resolution['720'] = True
  elif q.resolution == '1080p':
    resolution['1080'] = True
  elif q.resolution == '1440p':
    resolution['1440'] = True
  elif q.resolution == '2160p':
    resolution['2160'] = True


    
print("Available resolutions:")
for i in list(resolution.keys()):
  if resolution[i] != False: 
    print(i)

Available resolutions:
144
240
360
480


In [ ]:
x = True
y = True
while x == True:
  resolution_key = input("Enter a valid Resolution value: ") 
  for i in range(len(yt.streams)):

    if yt.streams[i].resolution == resolution_key+'p':
      x = False
      while y == True:
        key = input("To download as Video click --> v , To download as a mp3 click --> 3  :")
        if key == 'v':

          try:
            yt.streams[i].download()
            print("Video downloaded sucessfully")
            break
        
          except ValueError:
            print("Oopss..Link is not working")
            break
          y = False

        elif key == '3':    

          try:
            song = yt.streams[i].download()
            clip = mp.VideoFileClip(song).subclip(0,20)
            clip.audio.write_audiofile("theaudio.mp3")
            print("MP3 song downloaded sucessfully")
            break
        
          except ValueError:
            print("Oopss..Link is not working")
            break
          y = False

      
      break

  if x == True:    
    print("Oopss.. Please enter a valid resolution") 